### Используемые библиотеки

In [1]:
import pandas as pd
from skopt import BayesSearchCV
from skopt.space import Categorical
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [2]:
# считываем обработанные данные
df = pd.read_csv("data.csv")
df_y = df[['IC50, mM', 'CC50, mM', 'SI']]
df_x = df.drop(columns=['IC50, mM', 'CC50, mM', 'SI'])

### Регрессия SI

In [3]:
# датасет для сравнения метрик по каждой из модели
metrics = pd.DataFrame(columns=["Среднеквадратичная ошибка",
                                "Коэффициент детерминации",
                                "Коэффициент детерминации при обучение"]
                                )

In [4]:
# избавимся от лишних признаков, выделим целевую переменную и добавим дополнительный признак
y_data = df_y['SI']
x_data = df_x
x_data['CC50, mM'] = df_y['CC50, mM']

# сократим количество признаков до 50 с помощью Mutual Info
selector = SelectKBest(score_func=mutual_info_regression, k=50)
select_x_data = selector.fit_transform(x_data, y_data)
selected_features = x_data.columns[selector.get_support()]
x_data = pd.DataFrame(select_x_data, columns=selected_features)

# разделим данные на тестовыые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(f'Тренеровочные данные: {X_train.shape}, {y_train.shape}')
print(f'Tестовые данные: {X_test.shape}, {y_test.shape}')

# стандартизируем данные train
scaler = StandardScaler()
tmp_data = scaler.fit_transform(X_train)
scaled_X_train = pd.DataFrame(tmp_data, columns=x_data.columns)

# стандартизируем данные test
tmp_data = scaler.transform(X_test)
scaled_X_test = pd.DataFrame(tmp_data, columns=x_data.columns)

Тренеровочные данные: (772, 50), (772,)
Tестовые данные: (194, 50), (194,)


#### RandomForestRegressor

In [5]:
# определим пространство гиперпараметров для модели RandomForestRegressor
param_space_rfr = {
    'n_estimators': (50, 500),
    'max_depth': (1, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# создадим модель
model = RandomForestRegressor(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_rfr,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
r2_model = round(bayes_search.best_score_, 4)
print(f'R² = {r2_model}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
rfr_model = bayes_search.best_estimator_


R² = 0.3864

Лучшие параметры модели:
bootstrap = False
max_depth = 48
max_features = sqrt
min_samples_leaf = 1
min_samples_split = 4
n_estimators = 311


In [ ]:
# рассчитаем y
y_pred = rfr_model.predict(scaled_X_test)

# оценим метрики
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Среднеквадратичная ошибка: {round(rmse, 4)}")
print(f"Коэффициент детерминации: {round(r2, 4)}")

# сохраним данные в таблицу
res = {"Среднеквадратичная ошибка":round(rmse, 4),
       "Коэффициент детерминации": round(r2, 4),
       "Коэффициент детерминации при обучение": r2_model}
metrics.loc[0] = res

Среднеквадратичная ошибка: 1.2836
Коэффициент детерминации: 0.194


#### GradientBoostingRegressor

In [7]:
# определим пространство гиперпараметров для модели GradientBoostingRegressor
param_space_gbr = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 7),
}

# создадим модель
model = GradientBoostingRegressor(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_gbr,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
r2_model = round(bayes_search.best_score_, 4)
print(f'R² = {r2_model}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
gbr_model = bayes_search.best_estimator_

R² = 0.3643

Лучшие параметры модели:
max_depth = 19
min_samples_leaf = 5
min_samples_split = 9
n_estimators = 89


In [ ]:
# рассчитаем y
y_pred = gbr_model.predict(scaled_X_test)

# оценим метрики
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Среднеквадратичная ошибка: {round(rmse, 4)}")
print(f"Коэффициент детерминации: {round(r2, 4)}")

# сохраним данные в таблицу
res = {"Среднеквадратичная ошибка":round(rmse, 4),
       "Коэффициент детерминации": round(r2, 4),
       "Коэффициент детерминации при обучение": r2_model}
metrics.loc[1] = res

Среднеквадратичная ошибка: 1.2781
Коэффициент детерминации: 0.2009


#### KNeighborsRegressor

In [9]:
# определим пространство гиперпараметров для модели KNeighborsRegressor
param_space_knn = {
    'n_neighbors': (10, 100),
    'algorithm': Categorical(["auto", "brute"]),
    'leaf_size': (50, 200),
}

# создадим модель
model = KNeighborsRegressor()

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_knn,
    n_iter=20,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
r2_model = round(bayes_search.best_score_, 4)
print(f'R² = {r2_model}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
knn_model = bayes_search.best_estimator_

R² = 0.201

Лучшие параметры модели:
algorithm = brute
leaf_size = 50
n_neighbors = 10


In [ ]:
# рассчитаем y
y_pred = knn_model.predict(scaled_X_test)

# оценим метрики
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Среднеквадратичная ошибка: {round(rmse, 4)}")
print(f"Коэффициент детерминации: {round(r2, 4)}")

# сохраним данные в таблицу
res = {"Среднеквадратичная ошибка":round(rmse, 4),
       "Коэффициент детерминации": round(r2, 4),
       "Коэффициент детерминации при обучение": r2_model}
metrics.loc[2] = res

Среднеквадратичная ошибка: 1.3795
Коэффициент детерминации: 0.0691


In [11]:
metrics.index = ['RandomForestRegressor', 'GradientBoostingRegressor', 'KNeighborsRegressor']
metrics.head()

,Среднеквадратичная ошибка,Коэффициент детерминации,Коэффициент детерминации при обучение
RandomForestRegressor,1.2836,0.1940,0.3864
GradientBoostingRegressor,1.2781,0.2009,0.3643
KNeighborsRegressor,1.3795,0.0691,0.2010


#### Вывод
Для целевой переменной SI были дообработанны данные:
* Добавлен дополнительный признак CC50, mM в данные для обучения
* Сокращено количество признаков до 50
* Данные разделены на тестовыые и тренировочные
* Проведена стандартизация тренировочных данных и на её основе стандартизированы тестовые данные

Затем были подобраны параметры и построены три модели:

* RandomForestRegressor c параметрами:
    - bootstrap = False
    - max_depth = 48
    - max_features = sqrt
    - min_samples_leaf = 1
    - min_samples_split = 4
    - n_estimators = 311


* GradientBoostingRegressor с параметрами:
    - max_depth = 19
    - min_samples_leaf = 5
    - min_samples_split = 9
    - n_estimators = 89

* KNeighborsRegressor с параметрами:
    - algorithm = brute
    - leaf_size = 50
    - n_neighbors = 10

В качестве метрик для оценки работы модели использовались:
* Cреднеквадратичная ошибка (rmse)
* Коэффициент детерминации (R²)
* Коэффициент детерминации полученный при обучение

Наилучшие показатели и при обучение и при валидации на тестовых данных получились у модели RandomForestRegressor.